In [ ]:
import boto3
lambda_client = boto3.client("lambda")
s3_client = boto3.client("s3")

In [ ]:
import json

create_corpus_arn = ""

def create_corpus(name, s3_uri):

  json_data = { 
    "CorpusName": name,
    "S3Uri": s3_uri,
    "NMax": 5
  }
  
  payload = json.dumps({ "body": json.dumps(json_data) })

  response = lambda_client.invoke(
      FunctionName='',
      InvocationType='RequestResponse',
      Payload=payload
  )

  print(response)
  
  json_obj = json.loads(response['Payload'].read())
  data = json.loads(json_obj['body'])
  return data

In [ ]:
import json

compute_catchphrase_arn = ""

def compute_catchphrases(corpus_id, tag, top_k=100, affinity=0.10):

  json_data = { 
    "CorpusId": corpus_id, 
    "Tag": tag, 
    "TopK": top_k,
    "AffinityThreshold": affinity
  }
  
  payload = json.dumps({ "body": json.dumps(json_data) })

  response = lambda_client.invoke(
      FunctionName='',
      InvocationType='RequestResponse',
      Payload=payload
  )
  
  json_obj = json.loads(response['Payload'].read())
  data = json.loads(json_obj['body'])
  return data

In [ ]:
import time

sfn_client=boto3.client('stepfunctions')
def wait_for_sfn_sm(sm_execution_arn):
    status = 'RUNNING'
    while status == 'RUNNING':
        response = sfn_client.describe_execution(executionArn=sm_execution_arn)
        status = response.get('status')
        if status == 'RUNNING':
            time.sleep(15)
        
    return status

In [ ]:
def s3_bucket_keys(s3_client, bucket_name, bucket_prefix):
    """Generator for listing S3 bucket keys matching prefix"""

    kwargs = {'Bucket': bucket_name, 'Prefix': bucket_prefix}
    while True:
        resp = s3_client.list_objects_v2(**kwargs)
        for obj in resp['Contents']:
            yield obj['Key']

        try:
            kwargs['ContinuationToken'] = resp['NextContinuationToken']
        except KeyError:
            break

In [ ]:
s3_uris = []

universe_bucket=""
bucket_prefix = "memray/duc/documents/"

for key in s3_bucket_keys(s3_client=s3_client, bucket_name=universe_bucket, bucket_prefix=bucket_prefix):
    s3_uris.append(f"s3://{universe_bucket}/{key}")


In [ ]:
import re

sms = []
count = 0
max_count = 10000

filter_names = []
for s3_uri in s3_uris:
    m=re.match(r".+\/id=(\w+)\/.+", s3_uri)
    if m:
        name = f"duc-{m[1]}"
        if filter_names and name not in filter_names:
            continue
        response = create_corpus(name=name, s3_uri=s3_uri)
        sms.append( (name, response['CorpusStateMachine'],  response['CorpusId']) )
        count += 1
    if count >= max_count:
        break
print(f"Fast Corpus State Machines running count: {len(sms)}")

In [ ]:
create_corpus_failed = []
csms = []
top_k = 100
affinity = 0.10

for name, sm, corpus_id in sms:
    status = wait_for_sfn_sm(sm_execution_arn=sm)
    if status == "SUCCEEDED":
        response = compute_catchphrases(corpus_id=corpus_id, tag=name, top_k=top_k, affinity=affinity)
        csms.append( (name, response['CorpusStateMachine'], corpus_id) )
    else:
        create_corpus_failed.append((name, corpus_id))

if create_corpus_failed:
    print(f"Fast Corpus Failed: {create_corpus_failed}")

In [ ]:
retry_failed = [ name for name, _ in create_corpus_failed]
for s3_uri in s3_uris:
    m=re.match(r".+\/id=(\w+)\/.+", s3_uri)
    if m:
        name = f"duc-{m[1]}"
        if retry_failed and name not in retry_failed:
            continue
        response = create_corpus(name=name, s3_uri=s3_uri)
        sms.append( (name, response['CorpusStateMachine'],  response['CorpusId']) )
        

In [ ]:
compute_catchphrases_failed = []

for name, sm, corpus_id in csms:
    status = wait_for_sfn_sm(sm_execution_arn=sm)
    if status != "SUCCEEDED":
        compute_catchphrases_failed.append( (name, corpus_id) )

if compute_catchphrases_failed:
    print(f"Compute Catchphrases Failed: {compute_catchphrases_failed}")

In [ ]:
compute_catchphrases_failed.append( ('duc-270', '881d3e1e-581a-4327-9dcf-5eb6a3d4da43'))
compute_catchphrases_failed

In [ ]:
retry_sms = []
retry_failed = []
for name, corpus_id in compute_catchphrases_failed:
    response = compute_catchphrases(corpus_id=corpus_id, tag=name, top_k=top_k, affinity=affinity)
    retry_sms.append( (name, response['CorpusStateMachine'], corpus_id) )

for name, sm, corpus_id in retry_sms:
    status = wait_for_sfn_sm(sm_execution_arn=sm)
    if status != "SUCCEEDED":
        retry_failed.append( (name, corpus_id) )

if retry_failed:
    print(retry_failed)

In [ ]:
failed = []
while retry_failed:
    for name, corpus_id in retry_failed:
        response = compute_catchphrases(corpus_id=corpus_id, tag=name, top_k=top_k, affinity=affinity)
        sm = response['CorpusStateMachine']
        status = wait_for_sfn_sm(sm_execution_arn=sm)
        if status != "SUCCEEDED":
            failed.append( (name, corpus_id) )

    if failed:
        print(f"Failed: {failed}")
    retry_failed = failed

In [ ]:
from tempfile import NamedTemporaryFile
import gzip


corpora_bucket = ""

def get_extracted(name):
    bucket_prefix = f"catchphrases/tag={name}/"
        
    extracted = []
    try:
        for key in s3_bucket_keys(s3_client=s3_client, bucket_name=corpora_bucket, bucket_prefix=bucket_prefix):
            with NamedTemporaryFile(mode='w+b', delete=True) as file_obj:
                s3_client.download_fileobj(corpora_bucket, key, file_obj)
                file_obj.seek(0)

                with gzip.open(file_obj, mode="rb") as gzip_obj:
                    while (line := gzip_obj.readline()):
                        json_obj=json.loads(line.decode('utf-8'))
                        catchphrase = json_obj['catchphrase']
                        weight = json_obj['weight']
                        extracted.append( (catchphrase, weight) )
    except KeyError as e:
        print(e)

    extracted.sort(key = lambda x: x[1], reverse=True)
    return extracted


In [ ]:
import json

def get_gt_keyphrases(name):
    bucket_prefix = f"memray/duc/keyphrases/id={name}/"
        
    gt = []
    try:
        for key in s3_bucket_keys(s3_client=s3_client, bucket_name=universe_bucket, bucket_prefix=bucket_prefix):
            s3_obj = s3_client.get_object(Bucket=universe_bucket, Key=key)
            json_str = s3_obj['Body'].read().decode('utf-8')
            json_obj = json.loads(json_str)
            gt.extend(json_obj['keyphrases'])
    except KeyError as e:
        print(e)

    return gt

In [ ]:
import string

def normalize_phrases(retrieved, gt):
    normalized = retrieved

    for p in gt:
        i = 0
        while i < len(normalized):
            s = normalized[i]
            idx = s.find(p)
            if idx >= 0:
                s1 = s[:idx].strip()
                s2 = s[idx+len(p):].strip()
                insert_list = []
                if s1 and s1 not in string.punctuation:
                    insert_list.append(s1)
                insert_list.append(p)
                if s2 and s2 not in string.punctuation:
                    insert_list.append(s2)
                normalized = normalized[:i] + insert_list + normalized[i+1:]
                i = i + len(insert_list)
                break
            i+=1

    return normalized

In [ ]:
def f_score():

    true_positive = []
    false_positive = []
    false_negative = []

    for name, _, _ in sms:
        extracted = get_extracted(name)

        retrieved = []
        for phrase, _ in extracted:
            phrase = phrase.lower()
            retrieved.append(phrase)
        
        print(f"Retrieved: {retrieved}")
        gt_keyphrases = get_gt_keyphrases(name.rsplit('-', 1)[1])
        print(f"Ground Truth: {gt_keyphrases}")
        normalized = normalize_phrases(retrieved=retrieved, gt=gt_keyphrases)
        print(f"Normalized: {normalized}")

        tp = set()
        fn = set()
        for keyphrase in gt_keyphrases:
            if keyphrase in normalized:
                tp.add(keyphrase)
            else:
                fn.add(keyphrase)

        true_positive.append(len(tp))
        false_positive.append(len(normalized) - len(tp))
        false_negative.append(len(fn))
    

    tp_sum = sum(true_positive)
    fp_sum = sum(false_positive)
    fn_sum = sum(false_negative)

    precision = tp_sum / (tp_sum + fp_sum)
    recall = tp_sum / (tp_sum + fn_sum)
    F1 = 2*precision*recall/(precision+recall)
    F2 = 5*precision*recall/(4*precision+recall)

    return precision, recall, F1, F2
        

In [ ]:
import time

precision, recall, F1, F2 = f_score()

key = "FScore/duc270.json"
json_obj = {
    "dataset": "duc-270",
    "topk": top_k,
    "affinity": affinity,
    "timestamp": int(time.time()),
    "F1": F1,
    "F2": F2,
    "recall": recall,
    "precision": precision
}
print(json_obj)

s3_client.put_object( Bucket=corpora_bucket, Key=key, Body=json.dumps(json_obj))

    